# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob


In [4]:

# Write your code below.

pd_dir = os.getenv('PRICE_DATA')
pd_glob = glob(os.path.join(pd_dir, '**/*.parquet'),recursive= True)
df2 = dd.read_parquet(pd_glob).set_index("ticker")


In [5]:
df2.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, Date to Year
dtypes: datetime64[ns](1), object(1), float64(6), int32(1)

In [6]:
df2.head()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
ticker,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
# Write your code below.
df2_lag1= df2.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

df2_lag2= df2_lag1.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Adj_Close_lag_1 = x['Adj Close'].shift(1))
)

df_rets = df2_lag2.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

df_hi_lo = df_rets.assign(
    hi_lo_range = lambda x: x['High']- x['Low']
)


/var/folders/kb/wzn_7bf509s7f6q3b7yjdcnh0000gn/T/ipykernel_73831/2182863434.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df2_lag1= df2.groupby('ticker', group_keys=False).apply(
/var/folders/kb/wzn_7bf509s7f6q3b7yjdcnh0000gn/T/ipykernel_73831/2182863434.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df2_lag2= df2_lag1.groupby('ticker', group_keys=False).apply(


In [8]:
df_hi_lo

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
npartitions=90,,,,,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,object,int32,float64,float64,float64,float64
ACB,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZLAB,...,...,...,...,...,...,...,...,...,...,...,...,...
ZLAB,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
dd_feat = df_hi_lo.compute()
dd_feat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 336878 entries, A to ZLAB
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Date             336878 non-null  datetime64[ns]
 1   Open             336864 non-null  float64       
 2   High             336864 non-null  float64       
 3   Low              336864 non-null  float64       
 4   Close            336864 non-null  float64       
 5   Adj Close        336864 non-null  float64       
 6   Volume           336864 non-null  float64       
 7   source           336878 non-null  object        
 8   Year             336878 non-null  int32         
 9   Close_lag_1      336774 non-null  float64       
 10  Adj_Close_lag_1  336774 non-null  float64       
 11  Returns          336767 non-null  float64       
 12  hi_lo_range      336864 non-null  float64       
dtypes: datetime64[ns](1), float64(10), int32(1), object(1)
memory usage: 34.7+ MB


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

df2_m_avg= dd_feat.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Moving_avg = x['Returns'].rolling(10).mean())
)

In [11]:
df2_m_avg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 336878 entries, A to ZLAB
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Date             336878 non-null  datetime64[ns]
 1   Open             336864 non-null  float64       
 2   High             336864 non-null  float64       
 3   Low              336864 non-null  float64       
 4   Close            336864 non-null  float64       
 5   Adj Close        336864 non-null  float64       
 6   Volume           336864 non-null  float64       
 7   source           336878 non-null  object        
 8   Year             336878 non-null  int32         
 9   Close_lag_1      336774 non-null  float64       
 10  Adj_Close_lag_1  336774 non-null  float64       
 11  Returns          336767 non-null  float64       
 12  hi_lo_range      336864 non-null  float64       
 13  Moving_avg       335903 non-null  float64       
dtypes: datetime64[ns](1), float

In [13]:
df2_m_avg.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range,Moving_avg
ticker,,,,,,,,,,,,,,
ZLAB,2020-03-26,51.430000,52.790001,49.000000,52.590000,52.590000,431200.0,ZLAB.csv,2020,51.450001,51.450001,0.022157,3.790001,0.019330
ZLAB,2020-03-27,51.000000,51.705002,49.500000,50.959999,50.959999,415400.0,ZLAB.csv,2020,52.590000,52.590000,-0.030995,2.205002,0.001232
ZLAB,2020-03-30,51.529999,53.090000,50.748001,52.970001,52.970001,228000.0,ZLAB.csv,2020,50.959999,50.959999,0.039443,2.341999,0.018993
ZLAB,2020-03-31,53.250000,53.299999,50.174999,51.480000,51.480000,257800.0,ZLAB.csv,2020,52.970001,52.970001,-0.028129,3.125000,0.000396
ZLAB,2020-04-01,50.070000,51.910000,47.750000,48.820000,48.820000,316200.0,ZLAB.csv,2020,51.480000,51.480000,-0.051671,4.160000,0.000891


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it wasn't necessary, in fact it makes it slower.
Dask does these calculations faster, since it does parallel computation and lazy evaluations.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.